In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
import requests

def download_shakespeare():
    """Загружаем поэзию Шекспира"""
    url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [4]:
text = download_shakespeare().lower()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Уникальных символов: {vocab_size}")
print(f"Словарь: {''.join(chars)}")

# Создаем маппинги символ <-> индекс
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

Уникальных символов: 39
Словарь: 
 !$&',-.3:;?abcdefghijklmnopqrstuvwxyz


In [6]:


class ShakespeareLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_size, dropout):
        super(ShakespeareLSTM, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)

        self.rnn = nn.LSTM(input_size=emb_dim, hidden_size=hid_size)

        self.fc_out = nn.Linear(hid_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.n_layers=1
        self.hid_size = hid_size
        self.emb_dim = emb_dim


    def forward(self, x):
        batch_size = x.size(0)
        embedded = self.emb(x)

        output, hidden = self.rnn(embedded)
        output = self.dropout(output)
        output = self.fc_out(output)

        return output, hidden

    def init_hidden(self, batch_size, device):
        weight = next(self.parameters())
        return (weight.new_zeros(self.n_layers, batch_size, self.hid_size),
                weight.new_zeros(self.n_layers, batch_size, self.hid_size))



def train_model(model, train_loader, criterion, opt, n_epochs, device):
    ep_train_loss = []
    model.train(True)

    for i in range(n_epochs):
        train_loss = []
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output, _ = model(X_batch)

            loss = criterion(output.transpose(1, 2), y_batch)

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

            opt.step()
            opt.zero_grad()

            train_loss.append(loss.item())
        ep_train_loss.append(np.mean(train_loss))
        print(f"\nЭпоха {i+1}/{n_epochs} завершена:")
        print(f"  Средний Loss: {ep_train_loss[-1]:.4f}")

In [7]:
from torch.utils.data import Dataset, DataLoader

def encode(text):
    """Преобразует текст в индексы"""
    return [char_to_idx[ch] for ch in text]

def decode(indices):
    """Преобразует индексы в текст"""
    return ''.join([idx_to_char[idx] for idx in indices])

# Кодируем весь текст
data = torch.tensor(encode(text), dtype=torch.long)

class ShakespeareDataset(Dataset):
    def __init__(self, data, seq_length, max_samples=10000):
        self.data = data
        self.seq_length = seq_length
        self.max_samples = max_samples

    def __len__(self):
        return min(self.max_samples, len(self.data) - self.seq_length)

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + 1:idx + self.seq_length + 1]
        return x, y

# Параметры
SEQ_LENGTH = 100  # Длина последовательности для обучения
BATCH_SIZE = 64

# Создаем датасет и даталоадер
dataset = ShakespeareDataset(data, SEQ_LENGTH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

device = torch.device('cpu')
model = ShakespeareLSTM(vocab_size, emb_dim=128, hid_size=256, dropout=0.3).to(device)

criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters())
print('Starting Training')
train_model(model, dataloader, criterion, opt, 20, 'cpu')
print('Training finished successfully')


generated = generate_text(model, '', 180)
print(generated)

Starting Training

Эпоха 1/20 завершена:
  Средний Loss: 2.5585

Эпоха 2/20 завершена:
  Средний Loss: 2.3880

Эпоха 3/20 завершена:
  Средний Loss: 2.3776

Эпоха 4/20 завершена:
  Средний Loss: 2.3714

Эпоха 5/20 завершена:
  Средний Loss: 2.3676

Эпоха 6/20 завершена:
  Средний Loss: 2.3659

Эпоха 7/20 завершена:
  Средний Loss: 2.3632

Эпоха 8/20 завершена:
  Средний Loss: 2.3618

Эпоха 9/20 завершена:
  Средний Loss: 2.3598

Эпоха 10/20 завершена:
  Средний Loss: 2.3584

Эпоха 11/20 завершена:
  Средний Loss: 2.3573

Эпоха 12/20 завершена:
  Средний Loss: 2.3564

Эпоха 13/20 завершена:
  Средний Loss: 2.3555

Эпоха 14/20 завершена:
  Средний Loss: 2.3552

Эпоха 15/20 завершена:
  Средний Loss: 2.3540

Эпоха 16/20 завершена:
  Средний Loss: 2.3536

Эпоха 17/20 завершена:
  Средний Loss: 2.3524

Эпоха 18/20 завершена:
  Средний Loss: 2.3520

Эпоха 19/20 завершена:
  Средний Loss: 2.3515

Эпоха 20/20 завершена:
  Средний Loss: 2.3513
Training finished successfully


IndexError: index -1 is out of bounds for dimension 1 with size 0

In [12]:
def generate_text(model, start_str, length=500, temperature=0.8, device='cpu'):
    model.eval()

    # Если start_str пустой, начинаем с пробела или случайного символа
    if not start_str:
        start_str = ' '

    # Кодируем начальную строку
    indices = encode(start_str)

    generated = list(start_str)

    with torch.no_grad():
        # Подготавливаем начальный вход: (batch_size=1, seq_len=len(indices))
        input_seq = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)  # [1, seq_len]

        # "Разогреваем" модель на начальной последовательности
        if len(indices) > 1:
            # Пропускаем все кроме последнего символа
            _, hidden = model(input_seq[:, :-1])
            # Берем последний символ как начальный для генерации
            current_idx = input_seq[:, -1:]  # [1, 1]
        else:
            current_idx = input_seq  # [1, 1]
            hidden = None

        # Генерируем новые символы
        for _ in range(length):
            output, hidden = model(current_idx)

            logits = output[:, -1, :] / temperature

            probs = torch.softmax(logits, dim=-1)  # [1, vocab_size]

            next_idx = torch.multinomial(probs, num_samples=1)  # [1, 1]

            # Декодируем и добавляем
            next_char = idx_to_char[next_idx.item()]
            generated.append(next_char)

            # Обновляем вход для следующей итерации
            current_idx = next_idx.unsqueeze(0)  # [1, 1] (уже правильный размер)

    return ''.join(generated)

print(generate_text(model, ' ', 180))

ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead